In [21]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sns
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn import preprocessing
import math 
import json

In [19]:
test= {
    'KEYVALUES': "",               
    'RQUID': "",                     
    'RESULTID': "",                 
    'ERRORCODE': "",                  
    'ERRORDESC': "",               
    'IS_FRAUD': False
}
prediction_result=1


# Case OK
if(prediction_result==1):
    test['RESULTID']= "ProceedWithSuccess"                 
    test['ERRORCODE']= "00000"              
    test['ERRORDESC']= "PROCESSED_SUCCESSFULLY"              
    test['IS_FRAUD']= True

elif(prediction_result==0):
    test['RESULTID']= "ProceedWithSuccessMC"                 
    test['ERRORCODE']= "99999"              
    test['ERRORDESC']= "PROCESSED_UNSUCCESSFULLY "              
    test['IS_FRAUD']= False

else:
    test['RESULTID']= "SystemError"                 
    test['ERRORCODE']= "00001"              
    test['ERRORDESC']= "SYSTEM_ERROR"  



''

In [64]:
# Load the credit card transaction data
df = pd.read_excel(r'C:\Users\KABYADE\Desktop\Fraud_ML\dataset\clean_data.xlsx',dtype = str)

In [ ]:
def format_data(data):
    list=['V1','V2','V3','V4','V12','V13','V16','V19','V21','V22','V25' ,'V26','V27']
    df_back=df.filter(list)

    #Check for shape

    list=['V6','V7','V23','V8']
    #update_dates to d/m/y
    #remove_reste 
    #

In [65]:
def update_date(data):
    #DATETIME
    list=['V6','V7','V23','V8']
    date_columns=data.filter(list)
    year_columns = pd.DataFrame()
    
    for l in list:
        date_columns[l] = pd.to_datetime(date_columns[l])
        date_columns[l+'_month'] = date_columns[l].dt.month    
        date_columns[l+'_day'] = date_columns[l].dt.day.astype(int)
        year_columns[l+'_year'] = date_columns[l].dt.year

    date_columns.drop(list, axis=1, inplace=True)
    data.drop(list, axis=1, inplace=True)
   
    #Normalize date cols
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-math.pi, math.pi))
    date_columns_scaled = min_max_scaler.fit_transform(date_columns.values)
    date_columns_normalized = pd.DataFrame(date_columns_scaled, columns=date_columns.columns)
    date_columns_normalized = date_columns_normalized.applymap(lambda x: math.cos(x))

    new_df = pd.concat([data, date_columns_normalized, year_columns], axis=1)
    return new_df

def update_float(data):
    data['V10'] = data['V10'].astype(float)
    data['V9'] = data['V9'].astype(float)
    data['V24'] = data['V24'].astype(float)
    return data

def load_dict(col):
    with open('json/'+col+'_label_mapping.json', 'r') as f:
        label_mapping = json.load(f)
    return label_mapping

def update_object(data):
    list=['V1','V2','V3','V4','V12','V13','V16','V19','V21','V22','V25' ,'V26','V27']
    dt=data.filter(list)

    return data


In [77]:
data=[  '31', '281', '22',	'010112',	'010112', '6011', '21140121124C', '071181',	
        '01000002', '01000002',	'952',	'384',	'VERSUS BANK 2 PLATEAUX REGION LAGUNECI' ]
object_cols=['V1','V2','V3','V4','V12','V13','V16','V19','V21','V22','V25' ,'V26','V27']
i=0

for col in object_cols:
    mapping=load_dict(col)
    value=data[i]
    print(value," ===> ",mapping[value])
    data[i]=mapping[value]
    i=i+1

31  ===>  4
281  ===>  2
22  ===>  23
010112  ===>  1
010112  ===>  19
6011  ===>  2976
21140121124C  ===>  44
071181  ===>  20339
01000002  ===>  3
01000002  ===>  1
952  ===>  1
384  ===>  384
VERSUS BANK 2 PLATEAUX REGION LAGUNECI  ===>  4


In [67]:
def update_dtype():
    df['CLASS'] = df['CLASS'].astype(int)
    df['V10'] = df['V10'].astype(float)
    df['V9'] = df['V9'].astype(float)
    df['V24'] = df['V24'].astype(float)

# Object preprocessing 
def to_categorical(v_df):
    cols= v_df.select_dtypes(include=['object'])
    obj_cols=cols.columns.to_list()

    for col in obj_cols:
        le = LabelEncoder()
        temp_keys = v_df[col].values
        v_df[col] = le.fit_transform(v_df[col])
        label_mapping = dict(zip(temp_keys,v_df[col]))
        #print(label_mapping)
        
        with open('json/'+col+'_label_mapping.json', 'w') as f:
           json.dump(label_mapping, f)

    return v_df

def save_mapping(cls,data,col):
    label_mapping = dict(zip(cls, data))
    #print(label_mapping)
    with open('json/'+col+'_label_mapping.json', 'w') as f:
        json.dump(label_mapping, f)

def process_date():
    #DATETIME
    list=['V6','V7','V23','V8']
    date_columns=df.filter(list)
    year_columns = pd.DataFrame()
    
    for l in list:
        date_columns[l] = pd.to_datetime(date_columns[l])
        date_columns[l+'_month'] = date_columns[l].dt.month    
        date_columns[l+'_day'] = date_columns[l].dt.day.astype(int)
        year_columns[l+'_year'] = date_columns[l].dt.year

    date_columns.drop(list, axis=1, inplace=True)
    df.drop(list, axis=1, inplace=True)
   
    #Normalize date cols
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-math.pi, math.pi))
    date_columns_scaled = min_max_scaler.fit_transform(date_columns.values)
    date_columns_normalized = pd.DataFrame(date_columns_scaled, columns=date_columns.columns)
    date_columns_normalized = date_columns_normalized.applymap(lambda x: math.cos(x))

    new_df = pd.concat([df, date_columns_normalized, year_columns], axis=1)
    return new_df

In [ ]:
update_dtype()
df_final=process_date()
df_final=to_categorical(df_final)

In [183]:
df_final.to_excel('output/preprocessing_data.xlsx', index=False)

In [ ]:
#unique value per col
for col in df.columns:
    check=df[col].unique()
    print(str(col)+" ",len(check),check)

df.loc[df['CLASS'] == '0'].V6.describe()
# normal '010112', '101010'
# fraud '000010', '010112', '101010'